# Data Cleaning for Penda Health Center Dataset
## Preparing and Transforming Penda Health Center Data for Analysis
This comprehensive guide outlines the sequential process I followed to perform data cleaning on the dataset.
I have also uploaded a SQL File that has detailed steps on how I successfully integrated the three tables using Microsoft SQL Server and exported the combined results to a CSV file in MS Excel format. The SQL file also contains EDA that I did.

In [1]:
# Import Library and Dataset
import pandas as pd
import numpy as np

hospitals_df = pd.read_csv('D:/Data Analysis/Patient_Data/PendaHealth_Data.csv')
hospitals_df.head()
hospitals_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48147 entries, 0 to 48146
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   VisitCode      48147 non-null  object 
 1   PatientCode    48147 non-null  object 
 2   VisitDateTime  48147 non-null  object 
 3   Date           48147 non-null  object 
 4   Time           48147 non-null  object 
 5   MedicalCenter  48147 non-null  object 
 6   VisitCategory  48147 non-null  object 
 7   Payor          48147 non-null  object 
 8   NPS_Score      2022 non-null   float64
 9   Amount         48147 non-null  int64  
 10  Diagnosis      22119 non-null  object 
dtypes: float64(1), int64(1), object(9)
memory usage: 4.0+ MB


### Column Name and Definition
1. PatientCode: Which is a unique ID assigned to each Patient.
2. VisitDateTime:  Which is the date on which the visit occurred.
3. MedicalCenter: The medical centre at which the visit occurred.
4. Payor: The name of the payor for the visit, it can either be cash i.e., the person paid for themselves or it can be Insurance i.e., the bill was paid by an insurance company.
5. NPS_Score: Score provided by patient that rates their satisfaction level with the visit. Score range is 0-10.
6. Amount: The total amount invoiced for the visit.
7. Diagnosis: The nature of illness recorded for each visit.

Despite there being nun values in NPS_Score and Diagnosis, I won't bother to remove them because I won't be using NPS_Score in my Analysis. For the Null Diagnosis, I will Assume that there was non made that's why there are no entries.

First thing I will do is to check if there are duplicates. I will Use the 'PatientCode' Column becaus it is a Unique ID assigned to the patient.

In [42]:
# Check If There Are Duplicates Found in 'PatientCode' Column In The DataFrame named "hospitals_df"
# 'PatientCode' is  a Unique ID Assigned To Each Patient

duplicates = hospitals_df.duplicated(subset="PatientCode")
if duplicates.any():
    print("Duplicates found in the 'PatientCode' column.")
else:
    print("No duplicates found in the 'PatientCode' column.")

Duplicates found in the 'PatientCode' column.


* I will identify duplicate rows in the 'PatientCode' column in the DataFrame named & put it in a separate DataFrame.

In [ ]:
# Identify Duplicate Rows In The 'PatientCode' Column In The DataFrame Named "hospitals_df" & Put In A Separate DataFrame.
duplicate_rows2 = hospitals_df[hospitals_df.duplicated(subset="PatientCode", keep=False)]
duplicate_df2 = pd.DataFrame(duplicate_rows2)
duplicate_df2.head(20)
duplicate_df2.info()

# Save It As An Excel File
duplicate_df2.to_excel('D:/Data Analysis/Patient_Data/AccordingToPatientCode.xlsx')

### Results
After further exploration in Excel, I have discovered the following;
* There are Patients who visited the medical center more than once.
* There were duplicate entries in the 'VisitCode' Column. This means that there are patients who visited the hospital twice in the same Date and Time! Indicating that that there might have been errors in data entry. or This Patients were diagnosed with more than one disease on the same Date and Time. 

### Remedy
I decided to divide the dataset into teo dataframes namely;
1. DuplicateDiagnoses_df This dataframe contains information about patients who were diagnosed with multiple diseases or the same type of disease on the same 'VisitDateTime'.
2. UniqueDiagnoses_df This dataframe contains information about patients who were NOT diagnosed with one or more diseases on same 'VisitDateTime'.

In [50]:
# Extract information from the "duplicate_df2" about patients who were diagnosed with multiple diseases or the same type of disease on the same day and time,
duplicate_diagnoses = duplicate_df2[duplicate_df2.duplicated(subset=["PatientCode", "VisitDateTime", "Diagnosis"], keep=False)]
DuplicateDiagnoses_df = pd.DataFrame(duplicate_diagnoses)
DuplicateDiagnoses_df.head(10)
DuplicateDiagnoses_df.info()

# Save It As An Excel File
DuplicateDiagnoses_df.to_excel('D:/Data Analysis/Patient_Data/DuplicateDiagnoses_on_SamePatient.xlsx')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1636 entries, 24 to 48123
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   VisitCode      1636 non-null   object 
 1   PatientCode    1636 non-null   object 
 2   VisitDateTime  1636 non-null   object 
 3   Date           1636 non-null   object 
 4   Time           1636 non-null   object 
 5   MedicalCenter  1636 non-null   object 
 6   VisitCategory  1636 non-null   object 
 7   Payor          1636 non-null   object 
 8   NPS_Score      40 non-null     float64
 9   Amount         1636 non-null   int64  
 10  Diagnosis      561 non-null    object 
dtypes: float64(1), int64(1), object(9)
memory usage: 153.4+ KB


In [53]:
# Extract information from the "duplicate_df2" about patients who were not diagnosed with multiple diseases or the same type of disease on the same day and time.
unique_diagnoses = duplicate_df2.drop_duplicates(subset=["PatientCode", "VisitDateTime", "Diagnosis"], keep=False)
UniqueDiagnoses_df = pd.DataFrame(unique_diagnoses)
UniqueDiagnoses_df.head(10)
UniqueDiagnoses_df.info()

# Save It As An Excel File
UniqueDiagnoses_df.to_excel('D:/Data Analysis/Patient_Data/UniqueDiagnoses_on_SamePatient.xlsx')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27404 entries, 1 to 48143
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   VisitCode      27404 non-null  object 
 1   PatientCode    27404 non-null  object 
 2   VisitDateTime  27404 non-null  object 
 3   Date           27404 non-null  object 
 4   Time           27404 non-null  object 
 5   MedicalCenter  27404 non-null  object 
 6   VisitCategory  27404 non-null  object 
 7   Payor          27404 non-null  object 
 8   NPS_Score      1008 non-null   float64
 9   Amount         27404 non-null  int64  
 10  Diagnosis      14584 non-null  object 
dtypes: float64(1), int64(1), object(9)
memory usage: 2.5+ MB


### How I Fixed DataFrame 1 (DuplicateDiagnoses_df )
I will merge the duplicates based on the "VisitCode" column while summing the values in the "Amount" column only when they are not similar.
I have written two chunks of code where; 
* First one shows which rows in the 'Amount' column had different Values and which one has similar values.
* Second one produces the amount figures.

In [37]:
# To join the duplicate entries in the "DuplicateDiagnoses_on_SamePatient" DataFrame 
# based on the "VisitCode" column while summing the values in the "Amount" column only when they are not similar, 
# The following code will produce a dataframe that will show which row that was merged had two unique Diagnoses or similar Diagnosis

# Import the dataframe
duplicated_df = pd.read_excel('D:/Data Analysis/Patient_Data/DuplicateDiagnoses_on_SamePatient.xlsx')

# Joining duplicates based on VisitCode and summing Amount column based on condition
agg_funcs = {
    'PatientCode': 'first',
    'VisitDateTime': 'first',
    'Date': 'first',
    'Time': 'first',
    'MedicalCenter': 'first',
    'VisitCategory': 'first',
    'Payor': 'first',
    'NPS_Score': 'first',
    'Amount': lambda x: x.nunique(),
    'Diagnosis': 'first'
}

# Group by VisitCode and apply the aggregation functions
merged_df = duplicated_df.groupby('VisitCode').agg(agg_funcs).reset_index()

# Displaying the joined DataFrame
merged_df.head(50)


,VisitCode,PatientCode,VisitDateTime,Date,Time,MedicalCenter,VisitCategory,Payor,NPS_Score,Amount,Diagnosis
0,XA-1060353,651e3c41-2437,01/03/2022 09:15,01/03/2022,09:15:39,Githurai 45,In-person Visit,Insurance Company A,NaN,1,NaN
1,XA-1060401,06a8c2a1-ea4b,01/03/2022 10:12,01/03/2022,10:12:41,Mathare North,In-person Visit,Cash,NaN,1,NaN
2,XA-1060415,06a8c2a1-ea4b,01/03/2022 10:18,01/03/2022,10:18:03,Mathare North,In-person Visit,Insurance Company A,NaN,2,acute nasopharyngitis
3,XA-1060563,e6fc5e5e-c79b,01/03/2022 11:46,01/03/2022,11:46:11,Tassia,In-person Visit,Cash,NaN,1,NaN
4,XA-1060599,291d22fd-efc0,01/03/2022 12:02,01/03/2022,12:02:38,Mathare North,In-person Visit,Cash,NaN,1,NaN
5,XA-1060607,85f3d491-5e7e,01/03/2022 12:07,01/03/2022,12:07:32,Tassia,In-person Visit,Cash,NaN,1,NaN
6,XA-1060680,8464d445-dabd,01/03/2022 12:44,01/03/2022,12:44:41,Mathare North,In-person Visit,Cash,NaN,1,NaN
7,XA-1060764,b86918d6-3b16,01/03/2022 13:48,01/03/2022,13:48:54,Tassia,In-person Visit,Cash,NaN,1,NaN
8,XA-1061118,37d1da03-29fc,01/03/2022 18:42,01/03/2022,18:42:09,Kimathi Street,In-person Visit,Cash,NaN,2,NaN
9,XA-1061228,737d0f26-ab71,01/03/2022 20:37,01/03/2022,20:37:08,Embakasi,In-person Visit,Insurance Company A,NaN,1,"tonsillitis, acute bacterial"


In [61]:
# To join the duplicate entries in the "DuplicateDiagnoses_on_SamePatient" DataFrame 
# based on the "VisitCode" column while summing the values in the "Amount" column only when they are not similar, 
# This code will produce a dataframe that will show the amount after the duplicated entries have been merged.

# Import the dataframe
duplicated_df = pd.read_excel('D:/Data Analysis/Patient_Data/DuplicateDiagnoses_on_SamePatient.xlsx')

# Group by VisitCode and sum the Amount column while keeping other columns
grouped_df = duplicated_df.groupby('VisitCode').agg(
    {'PatientCode': 'first', 'VisitDateTime': 'first', 'Date': 'first', 'Time': 'first', 'MedicalCenter': 'first', 'VisitCategory': 'first',
     'Payor': 'first', 'NPS_Score': 'first', 'Amount': 'sum', 'Diagnosis': 'first'}
)

# Create a new DataFrame to store the merged entries
merged_df = pd.DataFrame(columns=grouped_df.columns)

# Iterate over each group in the grouped DataFrame
for visit_code, group in grouped_df.groupby('VisitCode'):
    # Check if there are duplicate entries
    if len(group) > 1:
        # Concatenate the unique values of the non-Amount columns and sum the Amount column
        merged_entry = group.iloc[0].copy()
        merged_entry['Amount'] = group['Amount'].sum()
        merged_df = merged_df.append(merged_entry)
    else:
        merged_df = merged_df.append(group)

# Reset the index of the merged DataFrame
merged_df.reset_index(drop=True, inplace=True)

# Displaying the merged DataFrame
merged_df.head(50)

# Save It As An Excel File
merged_df.to_excel('D:/Data Analysis/Patient_Data/MergedDuplicateDiagnoses_on_SamePatient.xlsx')

### Doing All The Above Using a Singular Code Block and Save Result In Excel
After employing the aforementioned code segments in a sequential manner to gain a comprehensive understanding of the data, I shall now proceed to consolidate the steps into a singular code block.  The objective is to eliminate the non-unique duplicate entries within the dataset and preserve the resulting data in a distinct Excel spreadsheet file termed "Clean_Data." Subsequently, the contents of the "merged_df" DataFrame will be replicated and pasted into the aforementioned file

In [60]:
# Import the dataframe
hospitals_df = pd.read_csv('D:/Data Analysis/Patient_Data/PendaHealth_Data.csv')

# Identify the non-unique duplicate rows
duplicate_diagnoses = hospitals_df[hospitals_df.duplicated(subset=["PatientCode", "VisitDateTime", "Diagnosis"], keep=False)]
DuplicateDiagnoses_df = pd.DataFrame(duplicate_diagnoses)

# Remove non-unique duplicate rows from the original data
unique_data = hospitals_df.drop_duplicates(subset=["PatientCode", "VisitDateTime", "Diagnosis"], keep=False)
UniqueData_df = pd.DataFrame(unique_data)

# Save It As An Excel File
UniqueData_df.to_excel('D:/Data Analysis/Patient_Data/Clean_Data.xlsx')